In [8]:
import argparse
import os
import sys
import pickle
import numpy as np
import networkx as nx

sys.stdout.flush()

num_workers = 2

# 将子目录添加到 sys.path
current_dir = os.getcwd()
sys.path.append(os.path.join(os.path.join(current_dir), 'EffectiveResistanceSampling'))
from EffectiveResistanceSampling.Network import *

sys.path.append(os.path.join(os.path.join(current_dir), 'utilities'))
from utilities.tools import *


def sparse_graph_mu(mu, graph_type, percent, epsilon=0.1):#, output_dir='graph_sparse'):
    output_dir = f'graph_sparse_{percent}'
    """Process a specific mixing parameter (mu) to get sparsed graphs."""
    graphs = load_graph_only(mu, graph_type, "original", percent)
    sample = len(graphs)
    if percent == 0.9:
        q_values = {"lfr": 46000, "ppm": 30000}
    elif percent == 0.6:
        q_values = {"lfr": 18000, "ppm": 12000} ### UNDER CONSTRUCTION!!!!!!!!!!

    graph_sparse = []

    for i in range(sample):
        G = graphs[i]

        edge_list = list(G.edges())
        edge_list = np.array(edge_list)

        edge_weights = nx.get_edge_attributes(G, 'weight')
        edge_weights = np.array(edge_weights)
        edge_weights = np.array([edge_weights[edge] if edge in edge_weights else 1 for edge in edge_list])

        Gn = Network(edge_list, edge_weights)
        Effective_R = Gn.effR(epsilon, 'spl')

        while True:
            Gn_Sparse = Gn.spl(q_values[graph_type], Effective_R, seed=2024)  # 第一个参数是 q 是边有放回抽样的数量
            G_sparse = to_networkx(Gn_Sparse)
            if nx.is_connected(G_sparse):
                break

        print(i)
        graph_sparse.append(G_sparse)

    # Save all graphs and memberships into a single file
    combined_data = {
        'graphs': graph_sparse,
    }

    os.makedirs(output_dir, exist_ok=True)

    mu_str = f"{mu:.2f}"
    file_path = os.path.join(output_dir, f'{graph_type}_graph_sparse_mu{mu_str}.pickle')
    with open(file_path, 'wb') as file:
        pickle.dump(combined_data, file)

    print(f'Saved all sparse graphs for mu={mu} to {file_path}')

graph_type = "ppm"
start_step = 0.05
percent = 0.6

if graph_type == "ppm":
    end_step = 0.9
elif graph_type == "lfr":
    end_step = 0.5
step_size = 0.05

mu = 0.05

sparse_graph_mu(mu, graph_type, percent)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Saved all sparse graphs for mu=0.05 to graph_sparse_0.6/ppm_graph_sparse_mu0.05.pickle
